In [5]:
import os

from langchain_community.document_loaders import PyPDFLoader

file_path = "BinanceUSDocs.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()
print(len(docs))

Current Working Directory: /Users/jamesspillmann/Desktop/aaa/projects/crypto_trader/bot/src/chain
219


In [8]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [9]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [10]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "How can I get the current price of bitcoin?"})

results

{'input': 'How can I get the current price of bitcoin?',
 'context': [Document(metadata={'page': 30, 'source': 'BinanceUSDocs.pdf'}, page_content='# Example A, symbol param provided\ncurl -X "GET" "https://api.binance.us/api/v3/ticker/24hr?symbol=BNBBTC"\n# Example B, no symbol provided\ncurl -X "GET" "https://api.binance.us/api/v3/ticker/24hr"\nResponse A\n{\n  "symbol": "BNBBTC",\n  "priceChange": "-0.00046450",\n  "priceChangePercent": "-4.659",\n  "weightedAvgPrice": "0.00978390",\n  "prevClosePrice": "0.00997050",\n  "lastPrice": "0.00950520",\n  "lastQty": "0.09000000",\n  "bidPrice": "0.00950060",\n  "bidQty": "0.25000000",\n  "askPrice": "0.00950520",\n  "askQty": "13.74000000",\n  "openPrice": "0.00996970",\n  "highPrice": "0.01012120",\n  "lowPrice": "0.00950500",\n  "volume": "7936.25000000",\n  "quoteVolume": "77.64747556",\n  "openTime": 1637042984994,\n  "closeTime": 1637129384994,\n  "firstId": 1851686,\n  "lastId": 1856703,\n  "count": 5018\n}\nResponse B\n[\n  {\n    "